# Set up

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json
import requests
from config import db_login_info

## Get NYT county COVID data

In [ ]:
# -----------------------
# This is a NYTimes report on national COVID-19 cases and deaths, sorted by county
# From: https://github.com/nytimes/covid-19-data (us-counties.csv)
# -----------------------

# Pull in the live data
counties_path = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"

# Save to a dataframe
covid_all_df = pd.read_csv(counties_path)

#### Build all values dataframe

In [ ]:
# Drop rows with null values (we want complete data only)
covid_all_df = covid_all_df.dropna()

# Convert fips to int
covid_all_df = covid_all_df.astype({'fips': 'int'})

In [ ]:
# Display the dataframe
#covid_all_df

#### Build latest values dataframe

In [ ]:
# Get the most up-to-date data
latest_date = covid_all_df['date'].iloc[-1]

# Make a dataframe holding that most recent data
covid_latest_df = covid_all_df.loc[covid_all_df['date']==latest_date]

In [ ]:
# Display the dataframe
#covid_latest_df

## Get mask use data

In [ ]:
# Import the mask CSV data
mask_path = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv"
masks_df = pd.read_csv(mask_path)

In [ ]:
# Clean up column names
masks_df.columns = ['fips','never','rarely','sometimes','frequently','always']

In [ ]:
# Change the null values (no data) to 0
masks_df = masks_df.fillna(0)

In [ ]:
# Display the dataframe
#masks_df

## Get CDC data
#### Source: https://data.cdc.gov/NCHS/Weekly-Counts-of-Deaths-by-State-and-Select-Causes/muzy-jte6

In [ ]:
cdc_url = 'https://data.cdc.gov/resource/muzy-jte6.json'
response = requests.get(cdc_url)
cdc_json = response.json()

# Convert to a dataframe
cdc_df = pd.DataFrame.from_dict(cdc_json)

In [ ]:
# Drop columns we don't need
cdc_df.drop(list(cdc_df)[19:34], axis=1,inplace=True)

# Rename columns
cdc_df.columns = ['state',
                  'year',
                  'week',
                  'week_ending_date',
                  'all_causes',
                  'natural_causes',
                  'septicemia',
                  'malignant_neoplasms',
                  'diabetes',
                  'alzheimers',
                  'influenza_and_pneumonia',
                  'chronic_lower_respiratory',
                  'other_diseases_of_respiratory',
                  'nephritis_nephrotic_syndrome',
                  'symptoms_signs_and_abnormal',
                  'diseases_of_heart',
                  'cerebrovascular_diseases',
                  'covid_19_multiple_causes',
                  'covid_19_underlying_cause']     


# Change the null values (no data) to 0
cdc_df = cdc_df.fillna(0)

In [ ]:
# Display the dataframe
#cdc_df

In [ ]:
# Build 2019 and 2020 dataframes, just in case
cdc_2019_df = cdc_df.loc[cdc_df['year']=="2019"]
cdc_2020_df = cdc_df.loc[cdc_df['year']=="2020"]

## Get Covid Tracking Project data
Source: https://covidtracking.com

#### Build latest values dataframe

In [2]:
covidtracking_current_url = 'https://api.covidtracking.com/v1/states/current.json'
covidtracking_current_response = requests.get(covidtracking_current_url)
covidtracking_current_json = covidtracking_current_response.json()

# Convert to a dataframe
covidtracking_current_df = pd.DataFrame.from_dict(covidtracking_current_json)

In [3]:
# Change the null values (no data) to 0
covidtracking_current_df = covidtracking_current_df.fillna(0)

In [4]:
# Remove columns we don't need
covidtracking_current_df.drop(list(covidtracking_current_df)[48:55], axis=1,inplace=True)
covidtracking_current_df.drop(list(covidtracking_current_df)[39], axis=1,inplace=True)
covidtracking_current_df.drop(list(covidtracking_current_df)[20], axis=1,inplace=True)
covidtracking_current_df.drop(list(covidtracking_current_df)[15:18], axis=1,inplace=True)

In [5]:
covidtracking_current_df.columns = ['date', 'state', 'positive', 'probable_cases', 'negative', 'pending',
       'total_test_results', 'hospitalized_currently', 'hospitalized_cumulative',
       'icu_currently', 'icu_cumulative', 'ventilator_currently',
       'ventilator_cumulative', 'recovered', 'data_quality_grade', 'deaths',
       'hospitalized', 'total_tests_viral', 'positive_tests_viral',
       'negative_tests_viral', 'positive_cases_viral', 'deaths_confirmed',
       'deaths_probable', 'total_test_encounters_viral', 'total_tests_people_viral',
       'total_tests_antibody', 'positive_tests_antibody', 'negative_tests_antibody',
       'total_tests_people_antibody', 'positive_tests_people_antibody',
       'negative_tests_people_antibody', 'total_tests_people_antigen',
       'positive_tests_people_antigen', 'total_tests_antigen',
       'positive_tests_antigen', 'positive_increase', 'negative_increase', 'total',
       'total_test_results_source', 'total_test_results_increase', 'pos_neg',
       'death_increase', 'hospitalized_increase']

In [6]:
# Display the dataframe
#covidtracking_current_df

#### Build all values dataframe

In [7]:
covidtracking_all_url = 'https://api.covidtracking.com/v1/us/daily.json'
covidtracking_all_response = requests.get(covidtracking_all_url)
covidtracking_all_json = covidtracking_all_response.json()

# Convert to a dataframe
covidtracking_all_df = pd.DataFrame.from_dict(covidtracking_all_json)

In [8]:
# Change the null values (no data) to 0
covidtracking_all_df = covidtracking_all_df.fillna(0)

In [9]:
# Remove columns we don't need
covidtracking_all_df.drop(list(covidtracking_all_df)[24], axis=1,inplace=True)
covidtracking_all_df.drop(list(covidtracking_all_df)[16], axis=1,inplace=True)
covidtracking_all_df.drop(list(covidtracking_all_df)[12], axis=1,inplace=True)

In [10]:
# Display the dataframe
covidtracking_all_df

,date,states,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,death,hospitalized,totalTestResults,total,posNeg,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,20201006,56,7460634,97932855,8680.0,31346.0,414461.0,6438.0,20973.0,1609.0,...,202675.0,414461.0,110226302,0,0,634,-624,722475,38661,823419
1,20201005,56,7421973,97210380,11544.0,30098.0,415085.0,6209.0,20812.0,1515.0,...,202041.0,415085.0,109402883,0,0,326,1519,846256,38133,935481
2,20201004,56,7383840,96364124,11471.0,29944.0,413566.0,5974.0,20729.0,1485.0,...,201715.0,413566.0,108467402,0,0,363,649,824848,38439,955736
3,20201003,56,7345401,95539276,11464.0,30106.0,412917.0,5996.0,20686.0,1501.0,...,201352.0,412917.0,107511666,0,0,741,1166,846390,51372,997984
4,20201002,56,7294029,94692886,10813.0,30697.0,411751.0,6106.0,20612.0,1533.0,...,200611.0,411751.0,106513682,0,0,835,1342,977186,49534,1127307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,20200126,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,0,0,0,0,0,0,0
255,20200125,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,0,0,0,0,0,0,0
256,20200124,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,0,0,0,0,0,0,0
257,20200123,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,0,0,0,0,0,0,1


In [11]:
covidtracking_all_df.columns

Index(['date', 'states', 'positive', 'negative', 'pending',
       'hospitalizedCurrently', 'hospitalizedCumulative', 'inIcuCurrently',
       'inIcuCumulative', 'onVentilatorCurrently', 'onVentilatorCumulative',
       'recovered', 'death', 'hospitalized', 'totalTestResults', 'total',
       'posNeg', 'deathIncrease', 'hospitalizedIncrease', 'negativeIncrease',
       'positiveIncrease', 'totalTestResultsIncrease'],
      dtype='object')

In [12]:
covidtracking_all_df.columns = ['date', 'states', 'positive', 'negative', 'pending',
       'hospitalized_currently', 'hospitalized_cumulative', 'icu_currently',
       'icu_cumulative', 'ventilator_currently', 'ventilator_cumulative',
       'recovered', 'deaths', 'hospitalized', 'total_test_results', 'total',
       'pos_neg', 'death_increase', 'hospitalized_increase', 'negative_increase',
       'positive_increase', 'total_test_results_increase']

In [13]:
covidtracking_all_df

,date,states,positive,negative,pending,hospitalized_currently,hospitalized_cumulative,icu_currently,icu_cumulative,ventilator_currently,...,deaths,hospitalized,total_test_results,total,pos_neg,death_increase,hospitalized_increase,negative_increase,positive_increase,total_test_results_increase
0,20201006,56,7460634,97932855,8680.0,31346.0,414461.0,6438.0,20973.0,1609.0,...,202675.0,414461.0,110226302,0,0,634,-624,722475,38661,823419
1,20201005,56,7421973,97210380,11544.0,30098.0,415085.0,6209.0,20812.0,1515.0,...,202041.0,415085.0,109402883,0,0,326,1519,846256,38133,935481
2,20201004,56,7383840,96364124,11471.0,29944.0,413566.0,5974.0,20729.0,1485.0,...,201715.0,413566.0,108467402,0,0,363,649,824848,38439,955736
3,20201003,56,7345401,95539276,11464.0,30106.0,412917.0,5996.0,20686.0,1501.0,...,201352.0,412917.0,107511666,0,0,741,1166,846390,51372,997984
4,20201002,56,7294029,94692886,10813.0,30697.0,411751.0,6106.0,20612.0,1533.0,...,200611.0,411751.0,106513682,0,0,835,1342,977186,49534,1127307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,20200126,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,0,0,0,0,0,0,0
255,20200125,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,0,0,0,0,0,0,0
256,20200124,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,0,0,0,0,0,0,0
257,20200123,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,0,0,0,0,0,0,1


## Get the county information table

In [ ]:
# Scrape the county info table from Wikipedia
county_url = 'https://en.wikipedia.org/wiki/User:Michael_J/County_table'
county_table = pd.read_html(county_url)

In [ ]:
# Grab the first table on the page and convert to dataframe
county_table_df = county_table[0]

# Drop columns we don't need
county_table_df = county_table_df.drop(columns=['Land Areakm²','Land Areami²','Water Areakm²','Water Areami²','Total Areakm²','Total Areami²','Sort [1]','Population(2010)'])

# Rename the columns
county_table_df.columns = ['state','fips','county','county_seat','lat','lon']

# Remove the + sign from latitude column
county_table_df['lat'] = county_table_df['lat'].str[1:]
county_table_df['lon'] = county_table_df['lon'].str[1:]

# Remove degree symbol from lat and lon
county_table_df['lat'] = county_table_df['lat'].str[:-1]
county_table_df['lon'] = county_table_df['lon'].str[:-1]

# Set as a float for lat and lon
county_table_df['lat'] = county_table_df['lat'].astype(float)
county_table_df['lon'] = county_table_df['lon'].astype(float)

# Convert the lon to an actual negative value (for all, since North America)
county_table_df['lon'] = -county_table_df['lon']

# Change the null values (no data) to 0
county_table_df = county_table_df.fillna(0)

In [ ]:
# Display the dataframe
#county_table_df

## Connect and load to postgressql
### Available dataframes:
<b>NYT:</b> covid_all_df, covid_latest_df <br>
<b>Masks:</b> masks_df <br>
<b>CDC:</b> cdc_df, cdc_2019_df, cdc_2020_df <br>
<b>Covid tracking:</b> covidtracking_current_df, covidtracking_all_df<br>
<b>County info:</b> county_table_df

In [14]:
# Connect to database
#
# Note: db_login_info comes from config.py and should have a scheme of:
# "postgresql://<USERNAME>:<PASSWORD>@<HOST>:<PORT>/<DATABASE_NAME>"

engine = create_engine(f'{db_login_info}')

In [15]:
# Check for Tables
table_names = engine.table_names()
table_names

['combined',
 'masks',
 'cdc',
 'county',
 'covid',
 'covidtracking_current',
 'covidtracking_all']

In [ ]:
engine.dispose()

### Load data into database

In [ ]:
# Load covid_all_df dataframe into database
covid_all_df.to_sql(name='covid', con=engine, if_exists='append', index=False)

In [ ]:
# Load mask_df dataframe into database
masks_df.to_sql(name='masks', con=engine, if_exists='append', index=False)

In [ ]:
# Load cdc_df dataframe into database
cdc_df.to_sql(name='cdc', con=engine, if_exists='append', index=False)

In [16]:
# Not yet working
covidtracking_current_df.to_sql(name='covidtracking_current', con=engine, if_exists='append', index=False)

In [17]:
# Not yet working
covidtracking_all_df.to_sql(name='covidtracking_all', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "deaths" of relation "covidtracking_all" does not exist
LINE 1: ...ator_currently, ventilator_cumulative, recovered, deaths, ho...
                                                             ^

[SQL: INSERT INTO covidtracking_all (date, states, positive, negative, pending, hospitalized_currently, hospitalized_cumulative, icu_currently, icu_cumulative, ventilator_currently, ventilator_cumulative, recovered, deaths, hospitalized, total_test_results, total, pos_neg, death_increase, hospitalized_increase, negative_increase, positive_increase, total_test_results_increase) VALUES (%(date)s, %(states)s, %(positive)s, %(negative)s, %(pending)s, %(hospitalized_currently)s, %(hospitalized_cumulative)s, %(icu_currently)s, %(icu_cumulative)s, %(ventilator_currently)s, %(ventilator_cumulative)s, %(recovered)s, %(deaths)s, %(hospitalized)s, %(total_test_results)s, %(total)s, %(pos_neg)s, %(death_increase)s, %(hospitalized_increase)s, %(nega

In [ ]:
# Load county_table_df dataframe into database
county_table_df.to_sql(name='county', con=engine, if_exists='append', index=False)

### Confirm data load

In [ ]:
# Confirm covid data has been added
pd.read_sql_query('select * from covid', con=engine)

In [ ]:
# Confirm mask use data has been added
pd.read_sql_query('select * from masks', con=engine)

In [ ]:
# Confirm cdc data has been added
pd.read_sql_query('select * from cdc', con=engine)

In [ ]:
# Confirm county data has been added
pd.read_sql_query('select * from county', con=engine)

In [ ]:
# Confirm covidtracking_all data has been added
pd.read_sql_query('select * from covidtracking_all', con=engine)

In [ ]:
# Confirm covid, mask, and county data has been joined
# Only do so after joining the tables using join_tables.sql
pd.read_sql_query('select * from combined', con=engine)